# CAFA 5 protein function Prediction with TensorFlow

In this notebook we trained a simple DNN model that predicts the function of the protein i.e., **GO term ID** using it's amino acid sequences.

In [ ]:
!pip install obonet
!pip install pyvis
!pip install biopython

## About the Dataset

### Protein Sequence

Each protein is composed of dozens or hundreds of amino acids that are linked sequentially. Each amino acid in the sequence may be represented by a one-letter or three-letter code. Thus the sequence of a protein is often notated as a string of letters.

# Gene Ontology

The functional properties of a proteins are defined by Gene Ontology(GO). Gene Ontology (GO) describes our understanding of the biological domain with respect to three aspects:
1. Molecular Function (MF)
2. Biological Process (BP)
3. Cellular Component (CC)

[Source](http://geneontology.org/docs/ontology-documentation).

# Dataset description

`train_sequences.fasta` :  contains the sequences for proteins with annotations (labelled proteins).

`train_terms.tsv` :  contains the list of annotated terms (ground truth) for the proteins in `train_sequences.fasta`. 

In `train_terms.tsv` the first column indicates the protein's UniProt accession ID (unique protein id), the second is the `GO Term ID`, and the third indicates in which ontology the term appears.

# Labels of the dataset

The objective of our model is to predict the terms (functions) of a protein sequence. One protein sequence can have many functions and can thus be classified into any number of terms. Each term is uniquely identified by a `GO Term ID`. Thus our model has to predict all the `GO Term ID`s for a protein sequence. Hence this is a multi-label classification problem.

# Protein embeddings for train and test data

To train a machine learning model we cannot use the alphabetical protein sequences in`train_sequences.fasta` directly. They have to be converted into a vector format. Hence we've converted the alphabetical protein sequences into numerical vector embeddings first and then train the model.

Protein sequences can be converted to vector embeddings using T5Tokenizer and T5Encoder from transformers available on hugging face. This results in a vector embedding of length 1024

# Import the Required Libraries

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import progressbar

# Read the data

In [ ]:
train_terms=pd.read_csv('/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv', sep="\t")
train_terms.head()
train_terms.shape

# Calculating protein embeddings

The folllowing code extracts the protein sequence from fasta file and extract embeddings of it using T5Tokenizer and T5Encoder imported from transformers. Each embedding is a vector of length 1024.

In [ ]:
import os
import json
from typing import Dict
from collections import Counter

import random
import obonet
import pandas as pd
import numpy as np
from Bio import SeqIO
import re
from transformers import T5Tokenizer, T5EncoderModel
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False) #.to(device)

# Load the model
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device)

def get_embeddings(seq):
    sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq)))]

    ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")

    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # generate embeddings
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,
                               attention_mask=attention_mask)

    # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7]) 
    emb_0 = embedding_repr.last_hidden_state[0]
    emb_0_per_protein = emb_0.mean(dim=0)
    
    return emb_0_per_protein

file = '/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta'
sequences = SeqIO.parse(file, "fasta")
print(next(iter(sequences)).seq)

In [ ]:
sequences = SeqIO.parse(file, "fasta")
get_embeddings(str(next(iter(sequences)).seq))

# Loading the protein embeddings


We can calculate the protein embeddings on our own, but the kaggle resources are not enough to collect embeddings for all the sequences in train dataset. So we are using the pre-calculated protein embeddings done by [Sergei Fironov](https://www.kaggle.com/sergeifironov) using the Rost Lab's T5 protein language model.

If the `t5embeds` is not yet added to the input data of the notebook, it can be added by clicking on `Add Data` and search for `t5embeds` and then click on the `+` beside it.

The protein embeddings to be used for training are recorded in `train_embeds.npy` and the corresponding protein ids are available in `train_ids.npy`.

We will load the protein ids of the protein embeddings in the train dataset contained in `train_ids.npy` into a numpy array.

In [ ]:
train_protein_ids = np.load('/kaggle/input/t5embeds/train_ids.npy')
train_protein_ids_pd=pd.DataFrame(train_protein_ids)
train_protein_ids_pd.head(100)

<!-- Now, we will load`train_embeds.py` which contains the pre-calculated embeddings of the proteins in the train dataset. with protein_ids (`id`s we loaded previously from the **train_ids.npy**) into a numpy array. This array now contains the precalculated embeddings for the protein_ids( Ids we loaded above from **train_ids.npy**) needed for training. -->

After loading the files as numpy arrays, we will convert them into Pandas dataframe.

Each protein embedding is a vector of length 1024. We create the resulting dataframe such that there are 1024 columns to represent the values in each of the 1024 places in the vector.

In [ ]:
train_embeddings = np.load('/kaggle/input/t5embeds/train_embeds.npy')
column_num = train_embeddings.shape[1]
train_df  = pd.DataFrame(train_embeddings, columns = ["Column_" + str(i) for i in range(1, column_num+1)])
train_df.head()

# Prepare the dataset

There are more than 40,000 labels in the `train_terms.tsv` file, so to simplify we will choose the most frequent 1500 `GO term IDs` as labels of the dataset.

In [ ]:
# Extracting Go IDs
num_of_labels = 1500
labels = train_terms['term'].value_counts().index
labels=labels[:num_of_labels]
labels

In [ ]:
# Extracting the dataset which contains the top 1500(labels) GO terms
train_terms_updated = train_terms.loc[train_terms['term'].isin(labels)]
train_terms_updated=train_terms_updated.reset_index(drop=True)
train_terms_updated

Plot the most frequent 100 `GO Term ID`s in `train_terms.tsv`.

In [ ]:
pie_df = train_terms_updated['aspect'].value_counts()
palette_color = sns.color_palette('bright')
plt.pie(pie_df.values, labels=np.array(pie_df.index), colors=palette_color, autopct='%.0f%%')
plt.show()

As evident from the pie chart above, majority of the `GO term Id`s have their aspect as BPO(Biological Process Ontology). In the labels array, absence or presence of each `GO term Id` is denoted by 0 or 1.

Uncomment the next 2 cells if labels are not already available in the datasets directory

In [ ]:
# train_size = train_protein_ids.shape[0]
# train_labels = np.zeros((train_size ,num_of_labels))
# series_train_protein_ids = pd.Series(train_protein_ids)

# bar = progressbar.ProgressBar(maxval=num_of_labels, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
# for i in range(num_of_labels):
#     n_train_terms = train_terms_updated[train_terms_updated['term'] ==  labels[i]]
#     label_related_proteins = n_train_terms['EntryID'].unique()
#     train_labels[:,i] =  series_train_protein_ids.isin(label_related_proteins).astype(float)
#     bar.update(i+1)
# bar.finish()

# labels_df = pd.DataFrame(data = train_labels, columns = labels)
# print(labels_df.shape)

`label_df` is composed of 1500 columns and 142246 entries.

In [ ]:
# #!mkdir /kaggle/working/labels
# labels_df = pd.DataFrame(data = train_labels, columns = labels)
# labels_df.to_csv('/kaggle/working/labels/kaggledata.csv')

Run the next cell only if labels are already available in the datasets directory

In [ ]:
labels_df=pd.read_csv('/kaggle/input/labels/kaggledata.csv')
labels_df=labels_df.drop(columns='Unnamed: 0')

In [ ]:
labels_df.shape

In [ ]:
labels_df.head()

# Training

We will use Tensorflow to train a Deep Neural Network with the protein embeddings to perform mult-label classification.

In [ ]:
train_df.shape

In [ ]:
labels_df.shape

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Split the data into train and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_df, labels_df, test_size=0.2, random_state=42)

# Define model architecture
INPUT_SHAPE = [train_df.shape[1]]
num_of_labels = labels_df.shape[1]

BATCH_SIZE = 5120

model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', tf.keras.metrics.AUC()]
)

# Train the model with validation split
history = model.fit(
    train_data,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20, # change this as needed recommended 100 epochs
    validation_data=(val_data, val_labels)
)

# Get the final training and validation metrics
final_train_metrics = model.evaluate(train_data, train_labels)
final_val_metrics = model.evaluate(val_data, val_labels)

print("Final Training Metrics:")
print("Loss:", final_train_metrics[0])
print("Binary Accuracy:", final_train_metrics[2])
print("AUC:", final_train_metrics[1])

print("\nFinal Validation Metrics:")
print("Loss:", final_val_metrics[0])
print("Binary Accuracy:", final_val_metrics[2])
print("AUC:", final_val_metrics[1])


In [ ]:
model.save("/kaggle/working/model1.h5")

In [ ]:
model.save_weights("/kaggle/working/model.weights.h5")

In [ ]:
model.load_weights("/kaggle/working/model.weights.h5")

In [ ]:
final_train_metrics = model.evaluate(train_data, train_labels)
final_val_metrics = model.evaluate(val_data, val_labels)

print("Final Training Metrics:")
print("Loss:", final_train_metrics[0])
print("Binary Accuracy:", final_train_metrics[1])
print("AUC:", final_train_metrics[2])

print("\nFinal Validation Metrics:")
print("Loss:", final_val_metrics[0])
print("Binary Accuracy:", final_val_metrics[1])
print("AUC:", final_val_metrics[2])


In [ ]:
custom_input_tensor = np.array([[1 for i in range(1024)]])
print(custom_input_tensor)
print(len(custom_input_tensor[0]))
# Get predictions for custom input tensor
predictions = model.predict(custom_input_tensor)

# 'predictions' will contain the model's output for the custom input tensor
print(predictions)
for i in predictions[0]:
    x=0 if i<0.5 else 1
    print(x)
# print(len(predictions))


# Plot the model's loss and accuracy for each epoch

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy']].plot(title="Accuracy")

# following is code for running inference

# we have added a test file too

In [ ]:
# !pip install gradio

In [ ]:
import tqdm
from Bio import SeqIO
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import json
from typing import Dict
from collections import Counter
import random
import obonet
from transformers import T5Tokenizer, T5EncoderModel
import torch
import re
# import gradio as gr

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_half_uniref50-enc', do_lower_case=False) #.to(device)

# Load the model
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_half_uniref50-enc").to(device)

def get_embeddings(seq):
    sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", seq)))]

    ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")

    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)

    # generate embeddings
    with torch.no_grad():
        embedding_repr = model(input_ids=input_ids,
                               attention_mask=attention_mask)

    # extract residue embeddings for the first ([0,:]) sequence in the batch and remove padded & special tokens ([0,:7])
    emb_0 = embedding_repr.last_hidden_state[0]
    emb_0_per_protein = emb_0.mean(dim=0)

    return emb_0_per_protein

def predict(fasta_file):
    sequences = SeqIO.parse(fasta_file, "fasta")

    ids = []
    num_sequences=sum(1 for seq in sequences)
    embeds = np.zeros((num_sequences, 1024))
    i = 0
    with open(fasta_file, "r") as fastafile:
      # Iterate over each sequence in the file
      for sequence in SeqIO.parse(fastafile, "fasta"):
        # Access the sequence ID and sequence data
        seq_id = sequence.id
        seq_data = str(sequence.seq)
        embeds[i] = get_embeddings(seq_data).detach().cpu().numpy()
#         print(embeds[i])
        ids.append(seq_id)
        i += 1
        
    INPUT_SHAPE=[1024]
    num_of_labels=1500

    model = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(units=num_of_labels, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['binary_accuracy', tf.keras.metrics.AUC()]
    )
    
    model.load_weights('/kaggle/working/model.weights.h5') #load model here
    labels_df=pd.read_csv('/kaggle/input/labels/kaggledata.csv')
    labels_df=labels_df.drop(columns='Unnamed: 0')

    predictions = model.predict(embeds)
    predictions_list1=[]
    predictions_list2=[]

    # 'predictions' will contain the model's output for the custom input tensor
    # print(predictions)
    for prediction in predictions:
        tmp=[]
        t2=[]
        for i in prediction:
            x=0 if i<0.4 else 1
            tmp.append(x)
            t2.append(i)
        predictions_list1.append(tmp.copy())
        predictions_list2.append(t2.copy())

    label_columns = labels_df.columns

    # Convert the predictions into a DataFrame
    predictions_df = pd.DataFrame(predictions_list1, columns=label_columns)
    p21=pd.DataFrame(predictions_list2, columns=label_columns)

    # Save the DataFrame to a CSV file
    predictions_df.to_csv("predictions.csv", index=False) #output csv
    p21.to_csv("decimal.csv",index=False)
    


In [ ]:
predict('/kaggle/input/fastaexample/example.fasta') #after this you will get a predictions.csv and deciaml.csv in the working directory